In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from estimators_get_pipe import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from setup_module.forestIV import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


Using MPS


0it [00:00, ?it/s]

Using MPS


<Figure size 640x480 with 0 Axes>

### Set variables

In [3]:
# Variables
method = 'Supervised'
classifiers_type = 'all'
if classifiers_type == 'nonlinear':
    classifiers_pipe = classifiers_pipe_nonlinear
elif classifiers_type == 'linear':
    classifiers_pipe = classifiers_pipe_linear
elif classifiers_type == 'ensemble':
    classifiers_pipe = classifiers_pipe_ensemble
elif classifiers_type == 'all':
    classifiers_pipe = classifiers_pipe

results_save_path = f'{models_save_path}{method} Results/'
with open(f'{data_dir}{method}_results_save_path.txt', 'w') as f:
    f.write(results_save_path)
if not os.path.exists(results_save_path):
    os.makedirs(results_save_path)
done_xy_save_path = f'{results_save_path}Search+Xy/'
with open(f'{data_dir}{method}_done_xy_save_path.txt', 'w') as f:
    f.write(done_xy_save_path)
if not os.path.exists(done_xy_save_path):
    os.makedirs(done_xy_save_path)

t = time.time()
n_jobs = -1
n_splits = 10
n_repeats = 3
random_state = 42
refit = True
class_weight = 'balanced'
cv = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
)
scoring = 'recall'
scores = [
    'recall', 'accuracy', 'f1', 'roc_auc',
    'explained_variance', 'matthews_corrcoef'
]
scorers = {
    'precision_score': make_scorer(precision_score, zero_division=0),
    'recall_score': make_scorer(recall_score, zero_division=0),
    'accuracy_score': make_scorer(accuracy_score, zero_division=0),
}
analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'
metrics_dict = {
    f'{scoring.title()} Best Score': np.nan,
    f'{scoring.title()} Best Threshold': np.nan,
    'Train - Mean Cross Validation Score': np.nan,
    f'Train - Mean Cross Validation - {scoring.title()}': np.nan,
    f'Train - Mean Explained Variance - {scoring.title()}': np.nan,
    'Test - Mean Cross Validation Score': np.nan,
    f'Test - Mean Cross Validation - {scoring.title()}': np.nan,
    f'Test - Mean Explained Variance - {scoring.title()}': np.nan,
    'Explained Variance': np.nan,
    'Accuracy': np.nan,
    'Balanced Accuracy': np.nan,
    'Precision': np.nan,
    'Average Precision': np.nan,
    'Recall': np.nan,
    'F1-score': np.nan,
    'Matthews Correlation Coefficient': np.nan,
    'Brier Score': np.nan,
    'Fowlkes–Mallows Index': np.nan,
    'R2 Score': np.nan,
    'ROC': np.nan,
    'AUC': np.nan,
    'Log Loss/Cross Entropy': np.nan,
    'Cohen’s Kappa': np.nan,
    'Geometric Mean': np.nan,
    'Classification Report': np.nan,
    'Imbalanced Classification Report': np.nan,
    'Confusion Matrix': np.nan,
    'Normalized Confusion Matrix': np.nan,
}

# Transformer variables
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available(
) else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
# Set random seed
random_state = 42
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
cores = multiprocessing.cpu_count()


Using MPS


### Functions

In [4]:
def save_df_full_summary_excel(
    df_full_summary,
    title,
    text_to_add_list,
    file_save_path,
    sheet_name=None,
    startrow=None,
    startcol=None,
):
    if sheet_name is None:
        sheet_name = 'All'
    if startrow is None:
        startrow = 1
    if startcol is None:
        startcol = 1

    # Define last rows and cols locs
    header_range = 1
    endrow = startrow + header_range + df_full_summary.shape[0]
    endcol = startcol + df_full_summary.shape[1]

    # Remove NAs
    df_full_summary = df_full_summary.fillna('')

    # Write
    writer = pd.ExcelWriter(f'{file_save_path}.xlsx')
    df_full_summary.to_excel(writer, sheet_name=sheet_name, merge_cells=True, startrow=startrow, startcol=startcol)
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    worksheet.set_column(startrow, 1, None, None, {'hidden': True}) # hide the index column

    # Title
    worksheet.merge_range(1, startcol, 1, endcol, title, workbook.add_format({'italic': True, 'font_name': 'Times New Roman', 'font_size': 12, 'font_color': 'black', 'align': 'left', 'top': True, 'bottom': True, 'left': False, 'right': False}))

    # Main body
    body_max_row_idx, body_max_col_idx = df_full_summary.shape

    for c, r in tqdm_product(range(body_max_col_idx), range(body_max_row_idx)):
        row_to_write = startrow + header_range + r
        col_to_write = startcol + 1 + c # 1 is for index
        body_formats = {'num_format': '0.00', 'font_name': 'Times New Roman', 'font_size': 12, 'font_color': 'black', 'align': 'center', 'text_wrap': True, 'left': False, 'right': False}

        if r == 0:
            body_formats |= {'top': True, 'bottom': True, 'left': False, 'right': False}
            worksheet.set_column(col_to_write, col_to_write, 10)

        if r == body_max_row_idx-1:
            body_formats |= {'bottom': True}

        if c == 0:
            body_formats |= {'align': 'left'}
            worksheet.set_column(col_to_write, col_to_write, 15)

        worksheet.write(row_to_write, col_to_write, df_full_summary.iloc[r, c], workbook.add_format(body_formats))

    # Add Note
    note_format = {'italic': True, 'font_name': 'Times New Roman', 'font_size': 10, 'font_color': 'black', 'align': 'left', 'left': False, 'right': False}
    worksheet.merge_range(endrow, startcol, endrow, endcol, 'Note.', workbook.add_format(note_format))
    # Add text
    for i, text in enumerate(text_to_add_list):
        worksheet.merge_range(endrow + 1 + i , startcol, endrow + 1 + i, endcol, text, workbook.add_format(note_format))

    writer.close()


In [5]:
def make_full_report(
    results, dv, dvs_name, dv_type,
    regression_info_dict=None, regressor_order=None, text_to_add_list=None, title=None, model_names=None
):
    '''
    Make a full report for a regression analysis.
    results: statsmodels regression results object or list of results objects
    dv: str, dependent variable name
    '''

    if regression_info_dict is None:
        # Regression info dict
        regression_info_dict = {
            'Model Name': lambda x: f'{x.model.__class__.__name__}',
            'N': lambda x: f'{int(x.nobs):d}',
            'R-squared': lambda x: f'{x.rsquared:.5f}',
            'R-squared Adj.': lambda x: f'{x.rsquared_adj:.5f}',
            'Log-Likelihood': lambda x: f'{x.llf:.5f}',
            'Pseudo R2': lambda x: f'{x.prsquared:.5f}',
            'F': lambda x: f'{x.fvalue:.5f}',
            'F (p-value)': lambda x: f'{x.f_pvalue:.5f}',
            'df_model': lambda x: f'{x.df_model:.0f}',
            'df_total': lambda x: f'{x.df_resid + x.df_model + 1:.0f}',
            'df_resid': lambda x: f'{x.df_resid:.0f}',
            'AIC': lambda x: f'{x.aic:.5f}',
            'BIC': lambda x: f'{x.bic:.5f}',
            'ICC': lambda x: f'{x.rsquared / (x.rsquared + (x.nobs - 1) * x.mse_resid):.5f}',
            'RMSE': lambda x: f'{x.mse_resid ** 0.5:.5f}',
            'RMSE (std)': lambda x: f'{x.mse_resid ** 0.5 / x.model.endog.std():.5f}',
            'Omnibus': lambda x: f'{sms.omni_normtest(x.resid).statistic:.5f}',
            'Omnibus (p-value)': lambda x: f'{sms.omni_normtest(x.resid).pvalue:.5f}',
            'Skew': lambda x: f'{sms.jarque_bera(x.resid)[-2]:.5f}',
            'Kurtosis': lambda x: f'{sms.jarque_bera(x.resid)[-1]:.5f}',
            'Jarque-Bera (JB)': lambda x: f'{sms.jarque_bera(x.resid)[0]:.5f}',
            'Jarque-Bera (p-value)': lambda x: f'{sms.jarque_bera(x.resid)[1]:.5f}',
            'Intercept': lambda x: f'{x.params["const"]:.5f}',
            'Intercept (std)': lambda x: f'{x.bse["const"]:.5f}',
            'Intercept t': lambda x: f'{x.tvalues["const"]:.5f}',
            'Intercept t (p-value)': lambda x: f'{x.pvalues["const"]:.5f}',
            'Intercept (95% CI)': lambda x: f'{x.conf_int().loc["const"][0]:.5f} - {x.conf_int().loc["const"][1]:.5f}',
            'Unstandardized Coefficent B (b)': lambda x: f'{x.params[0]:.5f}',
            'Standard Error (SE)': lambda x: f'{x.bse[0]:.5f}',
            'Standardized Coefficient b* (β)': lambda x: f'{x.params[0] / x.model.endog.std():.5f}',
            't': lambda x: f'{x.tvalues[0]:.5f}',
            't (p-value)': lambda x: f'{x.pvalues[0]:.5f}',
            '95% CI': lambda x: f'{x.conf_int().iloc[0, 1]:.5f} - {x.conf_int().iloc[0, 1]:.5f}',
            # 'Summary': lambda x: f'{x.summary()}',
            # 'F (p-value - FDR)': lambda x: f'{x.f_pvalue_fdr:.5f}',
            # 'F (p-value - Bonferroni)': lambda x: f'{x.f_pvalue_bonf:.5f}',
            # 't (p-value - FDR)': lambda x: f'{x.pvalues_fdr[1]:.5f}',
            # 't (p-value - Bonferroni)': lambda x: f'{x.pvalues_bonf[1]:.5f}',
        }
    if model_names is None:
        if isinstance(results, list):
            model_names = [
                f'{results[0].model.endog_names.split("_")[0] if "_" in results[0].model.endog_names else results[0].model.endog_names} Model {i}'
                for i in range(len(results[0].model.endog_names))
            ]
            model_names[0] = model_names[0].replace('Model 0', 'Full Model')
        else:
            model_names = [
                f'{results.model.endog_names.split("_")[0] if "_" in results.model.endog_names else results.model.endog_names}'
            ]

    order_type = 'unordered' if regressor_order is None else 'ordered'
    if text_to_add_list is None:
        text_to_add_list = []
        if regressor_order is not None:
            text_to_add_list.append('Models are ordered by independent variable type.')

        else:
            text_to_add_list.append('Models are ordered by coefficient size, largest to smallest.')

    if title is None:
        title = f'{dv_type} OLS Regression {dv}'

    try:
        # Statsmodels summary_col
        full_summary = summary_col(
            results,
            stars=True,
            info_dict=regression_info_dict,
            regressor_order=regressor_order,
            float_format='%0.3f',
            model_names=model_names,
        )
        if isinstance(results, list) and len(results) > 4:
            full_summary.tables[0][full_summary.tables[0].filter(regex='Full Model').columns[0]].loc['Unstandardized Coefficent B (b)': '95% CI'] = ''

        # Add title and notes
        full_summary.add_title(title)
        text_to_add_list.extend(full_summary.extra_txt)
        for text in text_to_add_list:
            full_summary.add_text(text)
        # Save
        save_name = f'{table_save_path}{title}'
        print(f'Saving {save_name}...')
        df_full_summary = pd.read_html(full_summary.as_html())[0]
        df_full_summary.to_csv(f'{save_name}.csv')
        df_full_summary.style.to_latex(f'{save_name}.tex', hrules=True)
        save_df_full_summary_excel(df_full_summary, title, text_to_add_list, save_name)

        return full_summary
    except IndexError:
        return None


In [6]:
def get_standardized_coefficients(results):

    # # Get standardized regression coefficients
    # std = np.asarray(constant.std(0))

    # if 'const' in results.params and 'const' in constant:
    #     std[0] = 1
    # tt = results.t_test(np.diag(std))
    # tt.c_names = results.model.exog_names

    # t-test
    std = results.model.exog.std(0)
    if 'const' in results.params:
        std[0] = 1
    tt = results.t_test(np.diag(std))
    if results.model.__class__.__name__ == 'MixedLM' or 'Group Var' in results.model.exog_names:
        offset = slice(None, -1)
        tt.c_names = results.model.exog_names[offset]
    else:
        offset = slice(None, None)
        tt.c_names = results.model.exog_names

    # Make df with standardized and unstandardized coefficients
    df_std_coef = pd.DataFrame(
        {
            'coef': results.params[offset].apply(lambda x: f'{x:.5f}'),
            'std err': results.bse[offset].apply(lambda x: f'{x:.5f}'),
            'std coef': (results.params[offset] / results.model.exog[offset].std(axis=0)).apply(lambda x: f'{x:.5f}'),
            't': results.tvalues[offset].apply(lambda x: f'{x:.5f}'),
            'P>|t|': results.pvalues[offset].apply(lambda x: f'{x:.5f}'),
            '[0.025': results.conf_int()[0][offset].apply(lambda x: f'{x:.5f}'),
            '0.975]': results.conf_int()[1][offset].apply(lambda x: f'{x:.5f}'),
        }
    )
    # if 'Group Var' in df_std_coef.index:
    #     df_std_coef = df_std_coef.drop('Group Var', axis='index')
    # # Add standardized coefficients and other data from t-test
    # df_std_coef['std coef'] = tt.effect
    # df_std_coef['std err'] = tt.sd
    # df_std_coef['t'] = tt.statistic
    # df_std_coef['P>|t|'] = tt.pvalue
    # df_std_coef['[0.025'] = tt.conf_int()[:, 0]
    # df_std_coef['0.975]'] = tt.conf_int()[:, 1]
    # df_std_coef['var'] = [names[i] for i in range(len(results.model.exog_names))]
    # df_std_coef = df_std_coef.sort_values('std coef', ascending=False)
    df_std_coef = df_std_coef.reset_index().rename(columns={'index': 'var'})
    df_std_coef = df_std_coef.rename(
        columns={
            'var': 'Variable',
            'coef': 'Unstandardized Coefficent B (b)',
            'std err': 'Standard Error',
            'std coef':'Standardized Coefficient b* (β)',
            't': 't-value',
            'P>|t|': 'p-value',
            '[0.025': '95% CI Lower',
            '0.975]': '95% CI Upper'
        }
    )
    # Reorder columns
    df_std_coef = df_std_coef[[
        'Variable',
        'Unstandardized Coefficent B (b)',
        'Standard Error',
        'Standardized Coefficient b* (β)',
        't-value',
        'p-value',
        '95% CI Lower',
        '95% CI Upper'
    ]]

    return tt, df_std_coef


In [7]:
# Function to compare and produce Unbiased and Biased OLS Models
def compare_actual_and_predicted(df, analysis_type, iv_names=None):
    dv_names_dict = defaultdict(lambda: defaultdict())

    for dv in tqdm.tqdm(dvs):
        if analysis_type == 'pre_classification':
            if iv_names is None:
                iv_names = ivs_dummy_perc_and_perc_interactions + controls[:2]
            dv_names_dict[dv] = {
                'Unbiased': {'dv_names': f'{dv}_actual'},
                'Biased': {'dv_names': f'{dv}_predicted'}
            }
            df = df.loc[
                (~df[dv_names_dict[dv]['Unbiased']['dv_names']].isna())
                & (~df[dv_names_dict[dv]['Biased']['dv_names']].isna())
            ]
            print(f'Processing dataframe of length {len(df)}')

        elif analysis_type == 'post_classification':
            if iv_names is None:
                iv_names = ivs_dummy_perc_and_perc_interactions[0]
            if f'{dv}_aggr_unlabeled_predicted' in df.columns:
                dv_names_dict[dv] = {
                    'Biased': {'dv_names': [f'{dv}_aggr_unlabeled_predicted'] + controls[:2]}
                }
                df = df.loc[
                    (~df[dv_names_dict[dv]['Biased']['dv_names'][0]].isna())
                ]
                print(f'Processing dataframe of length {len(df)}')
            elif f'{dv}_actual' in df.columns:
                dv_names_dict[dv] = {
                    'Unbiased': {'dv_names': [f'{dv}_actual'] + controls[:2]},
                }
                df = df.loc[
                    (~df[dv_names_dict[dv]['Unbiased']['dv_names'][0]].isna())
                ]
                print(f'Processing dataframe of length {len(df)}')

        print(f'Analyzing {dv} {dv_names_dict[dv].keys()} Models')

        for dv_type, dv_names in tqdm.tqdm(dv_names_dict[dv].items()):
            if analysis_type == 'pre_classification':
                endog = df[dv_names['dv_names']]
                exog = df[iv_names]
            elif analysis_type == 'post_classification':
                endog = df[iv_names]
                exog = df[dv_names['dv_names']]

            constant = sm.add_constant(exog)
            model = sm.OLS(endog=endog, exog=constant, data=df)
            results = model.fit()
            tt, df_std_coef = get_standardized_coefficients(results)
            title = f'{analysis_type} {dv_type} OLS Regression {dv}'
            full_summary = make_full_report(
                results, dv, dvs_name=dv, dv_type=dv_type, title=title
            )
            dv_names_dict[dv][dv_type]['R-squared'] = results.rsquared
            dv_names_dict[dv][dv_type]['Results'] = results

            print('\n')
            print('-'*20)
            print(f'{dv_type.upper()} {dv}\n')
            print('-'*20)
            print('\n')
            print(f'{dv_type.upper()} SUMMARY RESULTS:')
            print(results.summary())
            print(full_summary)
            print('\n')
            print('-'*20)
            print(f'{dv_type.upper()} STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
            print('\n')
            print('-'*20)

            save_name = f'{table_save_path}{title}'
            df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))
            df_summary_results.to_csv(f'{save_name}.csv')
            df_summary_results.style.to_latex(f'{save_name}.tex', hrules=True)
            df_std_coef.to_csv(f'{save_name} - standardized coefficients.csv')
            df_std_coef.style.to_latex(f'{save_name} - standardized coefficients.tex', hrules=True)

        if dv_names_dict[dv][list(dv_names_dict[dv])[0]]['R-squared'] != dv_names_dict[dv][list(dv_names_dict[dv])[-1]]['R-squared']:
            print('\n')
            print('-'*20)
            print(f'{dv} {list(dv_names_dict[dv])[0]} R-Squared does not equal {list(dv_names_dict[dv])[-1]} R-Squared:')
            print(f'{dv} {list(dv_names_dict[dv])[0]} = {dv_names_dict[dv][list(dv_names_dict[dv])[0]]["R-squared"]:.3f}')
            print(f'{dv} {list(dv_names_dict[dv])[-1]} = {dv_names_dict[dv][list(dv_names_dict[dv])[-1]]["R-squared"]:.3f}')
            print('\n')
            print('-'*20)

    return dict(dv_names_dict)


### READ DATA

In [8]:
with open(f'{data_dir}df_jobs_for_analysis_len.txt', 'r') as f:
    df_jobs_len = int(f.read())

df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_analysis.pkl')
assert len(df_jobs) == df_jobs_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'
print(f'Dataframe df_jobs_for_analysis loaded with shape: {df_jobs.shape}')


Dataframe df_jobs_for_analysis loaded with shape: (309438, 79)


In [9]:
df_jobs['Warmth'].equals(df_jobs['Warmth_predicted'])


False

In [10]:
df_jobs['Competence'].equals(df_jobs['Competence_predicted'])


False

In [11]:
df_jobs[['Warmth', 'Warmth_actual']].isna().sum()


Warmth                0
Warmth_actual    303494
dtype: int64

## Check biased and unbiased regressions models using human annotated and classifier predicted Warmth and Competence
Source: https://mochenyang.github.io/mochenyangblog/research/2022/01/10/ForestIV.html

### Unbiased and Biased Warmth and CompetenceOLS regression with human annotated actual values as DV and all IVs

In [12]:
dv_names_dict_pre_classification = compare_actual_and_predicted(df_jobs, analysis_type='pre_classification')


  0%|          | 0/2 [00:00<?, ?it/s]

Processing dataframe of length 5944
Analyzing Warmth dict_keys(['Unbiased', 'Biased']) Models


Saving /Users/nyxinsane/Documents/Work - UvA/Automating Equity/Automating_Equity1/Automating_Equity1_Code/data/output tables/pre_classification Unbiased OLS Regression Warmth...


  0%|          | 0/140 [00:00<?, ?it/s]



--------------------
UNBIASED Warmth

--------------------


UNBIASED SUMMARY RESULTS:


...



--------------------
UNBIASED Competence

--------------------


UNBIASED SUMMARY RESULTS:
                            OLS Regression Results                            
Dep. Variable:      Competence_actual   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     48.09
Date:                Wed, 01 Nov 2023   Prob (F-statistic):          3.64e-127
Time:                        04:38:44   Log-Likelihood:                -3980.0
No. Observations:                5944   AIC:                             7990.
Df Residuals:                    5929   BIC:                             8090.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
-----

Saving /Users/nyxinsane/Documents/Work - UvA/Automating Equity/Automating_Equity1/Automating_Equity1_Code/data/output tables/pre_classification Biased OLS Regression Competence...


  0%|          | 0/140 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  2.36it/s]



--------------------
BIASED Competence

--------------------


BIASED SUMMARY RESULTS:
                             OLS Regression Results                             
Dep. Variable:     Competence_predicted   R-squared:                       0.124
Model:                              OLS   Adj. R-squared:                  0.122
Method:                   Least Squares   F-statistic:                     60.18
Date:                  Wed, 01 Nov 2023   Prob (F-statistic):          3.00e-159
Time:                          04:38:44   Log-Likelihood:                -3917.1
No. Observations:                  5944   AIC:                             7864.
Df Residuals:                      5929   BIC:                             7965.
Df Model:                            14                                         
Covariance Type:              nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025  

In [13]:
dv_names_dict_pre_classification


{'Warmth': {'Unbiased': {'dv_names': 'Warmth_actual',
   'R-squared': 0.07789376529661218,
   'Results': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x29fc27ee0>},
  'Biased': {'dv_names': 'Warmth_predicted',
   'R-squared': 0.0937257615180127,
   'Results': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x29f6cf880>}},
 'Competence': {'Unbiased': {'dv_names': 'Competence_actual',
   'R-squared': 0.10197425136976457,
   'Results': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x29fc271f0>},
  'Biased': {'dv_names': 'Competence_predicted',
   'R-squared': 0.1244277453998145,
   'Results': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x29fc26d40>}}}

## Make RandomForestRegressor Classifier


In [14]:
def make_final_indiv_and_aggr_preds(estimator, X):
    pred = estimator.predict(X)
    indiv_pred = [tree.predict(X) for tree in estimator.estimators_]
    aggr_pred = np.mean(indiv_pred, axis=0)

    return pred, indiv_pred, aggr_pred


In [15]:
def get_randomforest_instrumental_variable_estimator(df_jobs, cols_to_compare=None, text_col=None, n_trees=None):

    if cols_to_compare is None:
        cols_to_compare = ['Warmth_actual', 'Warmth_predicted', 'Competence_actual', 'Competence_predicted']
    if text_col is None:
        text_col = 'Job Description spacy_sentencized'
    if n_trees is None:
        n_trees = 100
    cols_dict = defaultdict()
    train_ratio = 0.75
    test_ratio = 0.10
    validation_ratio = 0.15
    test_split = test_size = 1 - train_ratio
    validation_split = test_ratio / (test_ratio + validation_ratio)

    # Make df_jobs_unlabeled
    df_jobs_unlabeled = df_jobs.loc[
        (df_jobs[cols_to_compare].isna()).all(axis='columns')
    ]

    if all(df_jobs_unlabeled.isna().sum()) != 0:
        raise IndexError('Missing data in df_jobs_labeled.')

    df_jobs_unlabeled = df_jobs_unlabeled.dropna(subset=analysis_columns, how='any')
    print(f'Dataframe df_jobs_unlabeled of length: {len(df_jobs_unlabeled)}')

    # Make df_jobs_labeled
    df_jobs_labeled = df_jobs.loc[
        (~df_jobs[cols_to_compare].isna()).all(axis='columns')
    ]

    if all(df_jobs_labeled.isna().sum()) != 0:
        raise IndexError('Missing data in df_jobs_labeled.')

    df_jobs_labeled = df_jobs_labeled.dropna(subset=analysis_columns, how='any')
    print(f'Dataframe df_jobs_labeled of length: {len(df_jobs_labeled)}')

    # Make df labels dict
    df_add_preds_dict = {
        'labeled': df_jobs_labeled,
        'unlabeled': df_jobs_unlabeled
    }

    # Split data
    print('Splitting data...')
    train, test = train_test_split(
        df_jobs_labeled, train_size=1-test_split, test_size=test_split, random_state=random_state
    )
    print(f'Length of train dataset: {len(train)}')
    print(f'Length of test dataset: {len(test)}')
    cols_dict = {
        'train': train, 'test': test,
    }

    for col in tqdm.tqdm(analysis_columns):
        assert col in df_jobs_labeled.columns, f'{col} column not found in df_jobs_labeled'
        print('='*20)
        print(f'Training on {col}...')

        X_train = np.array(list(train[text_col].astype('str').values))
        y_train = column_or_1d(train[col].astype('int64').values.tolist(), warn=True)

        X_test = np.array(list(test[text_col].astype('str').values))
        y_test = column_or_1d(test[col].astype('int64').values.tolist(), warn=True)

        X_labeled = np.array(list(df_jobs_labeled[text_col].astype('str').values))
        y_labeled = column_or_1d(df_jobs_labeled[col].astype('int64').values.tolist(), warn=True)

        X_unlabeled = np.array(list(df_jobs_unlabeled[text_col].astype('str').values))
        y_unlabeled = column_or_1d(df_jobs_unlabeled[col].astype('int64').values.tolist(), warn=True)

        # Vectorize using FeatueUnion
        print(f'Vectorizing using {vectorizers_list[-1].__class__.__name__}...')
        vectorizer = vectorizers_list[-1]
        X_train = vectorizer.fit_transform(X_train)
        X_test = vectorizer.transform(X_test)
        X_labeled = vectorizer.transform(X_labeled)
        X_unlabeled = vectorizer.transform(X_unlabeled)

        # Train using RandomForestRegressor
        print('Training using RandomForestRegressor...')
        estimator = RandomForestRegressor(n_estimators=n_trees, random_state=random_state, n_jobs=n_jobs)
        estimator.fit(X_train, y_train)

        # Get predictions
        print('Getting predictions...')
        y_train_pred, indiv_y_train_pred, aggr_y_train_pred = make_final_indiv_and_aggr_preds(estimator, X_train)
        y_test_pred, indiv_y_test_pred, aggr_y_test_pred = make_final_indiv_and_aggr_preds(estimator, X_test)
        y_labeled_pred, indiv_y_labeled_pred, aggr_y_labeled_pred = make_final_indiv_and_aggr_preds(estimator, X_labeled)
        y_unlabeled_pred, indiv_y_unlabeled_pred, aggr_y_unlabeled_pred = make_final_indiv_and_aggr_preds(estimator, X_unlabeled)

        # Make col dict
        cols_dict[col] = {
            'estimator': estimator, 'vectorizer': vectorizer,
            'X_train': X_train, 'y_train': y_train, 'y_train_pred': y_train_pred,
            'indiv_y_train_pred': indiv_y_train_pred, 'aggr_y_train_pred': aggr_y_train_pred,
            'X_test': X_test, 'y_test': y_test, 'y_test_pred': y_test_pred,
            'indiv_y_test_pred': indiv_y_test_pred, 'aggr_y_test_pred': aggr_y_test_pred,
            'X_labeled': X_labeled, 'y_labeled': y_labeled, 'y_labeled_pred': y_labeled_pred,
            'indiv_y_labeled_pred': indiv_y_labeled_pred, 'aggr_y_labeled_pred': aggr_y_labeled_pred,
            'X_unlabeled': X_unlabeled, 'y_unlabeled': y_unlabeled, 'y_unlabeled_pred': y_unlabeled_pred,
            'indiv_y_unlabeled_pred': indiv_y_unlabeled_pred, 'aggr_y_unlabeled_pred': aggr_y_unlabeled_pred,
        }

        # Add columns to df
        for df_lab, df in tqdm.tqdm(df_add_preds_dict.items()):
            df = pd.concat(
                [
                    df.reset_index(drop=True),
                    pd.DataFrame(
                        {
                            f'{col}_{df_lab}_predicted': cols_dict[col][f'y_{df_lab}_pred'],
                            f'{col}_aggr_{df_lab}_predicted': cols_dict[col][f'aggr_y_{df_lab}_pred'],
                        }
                    ).reset_index(drop=True),
                    pd.DataFrame(cols_dict[col][f'indiv_y_{df_lab}_pred']).transpose().add_prefix(f'{col}_tree_').reset_index(drop=True)
                ],
                axis='columns'
            )
            cols_dict[col][f'df_jobs_{df_lab}'] = df

        # Evaluate
        print('Evaluating...')
        score = estimator.score(X_test, y_test)
        mae = mean_absolute_error(y_test, y_test_pred)
        mse = mean_squared_error(y_test, y_test_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_test_pred)

        print('-'*20)
        print(f'Mean Absolute Error: {mae:3f}')
        print(f'Mean Squared Error: {mse:3f}')
        print(f'Root Mean Squared Error: {rmse:3f}')
        print(f'R-squared (R^2) Score: {r2:3f}')
        print('-'*20)

    return n_trees, dict(cols_dict)


In [16]:
n_trees, cols_dict = get_randomforest_instrumental_variable_estimator(df_jobs)


Dataframe df_jobs_unlabeled of length: 303494
Dataframe df_jobs_labeled of length: 5944
Splitting data...
Length of train dataset: 4458
Length of test dataset: 1486


  0%|          | 0/2 [00:00<?, ?it/s]

Training on Warmth...
Vectorizing using FeatureUnion...
Training using RandomForestRegressor...
Getting predictions...


 50%|█████     | 1/2 [00:31<00:31, 31.10s/it]

Evaluating...
--------------------
Mean Absolute Error: 0.323022
Mean Squared Error: 0.177012
Root Mean Squared Error: 0.420728
R-squared (R^2) Score: 0.066725
--------------------
Training on Competence...
Vectorizing using FeatureUnion...
Training using RandomForestRegressor...
Getting predictions...


100%|██████████| 2/2 [01:00<00:00, 30.23s/it]

Evaluating...
--------------------
Mean Absolute Error: 0.394408
Mean Squared Error: 0.210227
Root Mean Squared Error: 0.458505
R-squared (R^2) Score: 0.145708
--------------------


In [17]:
cols_dict.keys()


dict_keys(['train', 'test', 'Warmth', 'Competence'])

In [18]:
list_columns = [c for c in df_jobs.columns if df_jobs[c].apply(lambda x: isinstance(x, list)).any()]
non_list_columns = [c for c in df_jobs.columns if not df_jobs[c].apply(lambda x: isinstance(x, list)).any()]


In [19]:
cols_dict['Warmth']['df_jobs_labeled'].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5944 entries, 0 to 5943
Columns: 181 entries, Search Keyword to Warmth_tree_99
dtypes: category(2), float64(158), int64(2), object(19)
memory usage: 8.1+ MB


In [20]:
cols_dict['Competence']['df_jobs_labeled'].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5944 entries, 0 to 5943
Columns: 181 entries, Search Keyword to Competence_tree_99
dtypes: category(2), float64(158), int64(2), object(19)
memory usage: 8.1+ MB


In [21]:
df_jobs_labeled = cols_dict['Warmth']['df_jobs_labeled']\
    .drop(columns=list_columns)\
    .merge(
        cols_dict['Competence']['df_jobs_labeled'],
        how='outer',
        on=non_list_columns
    ).dropna(axis='columns', how='all')\
        .reset_index(drop=True)


In [22]:
df_jobs_labeled.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5944 entries, 0 to 5943
Columns: 283 entries, Search Keyword to Competence_tree_99
dtypes: category(2), float64(260), int64(2), object(19)
memory usage: 12.8+ MB


In [23]:
df_jobs_labeled.head()


,Search Keyword,Platform,Job ID,Job Title,Company Name,Location,Dutch Requirement in Job Ad,English Requirement in Job Ad,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Sector Code,Sector,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description spacy_sentencized_lower,Dutch Requirement in Sentence,English Requirement in Sentence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Job Description spacy_sentencized_cleaned,Warmth,Warmth_Probability,Competence,Competence_Probability,Warmth_actual,Competence_actual,Warmth_predicted,Warmth_Probability_predicted,Competence_predicted,Competence_Probability_predicted,Warmth_labeled_predicted,Warmth_aggr_labeled_predicted,Warmth_tree_0,Warmth_tree_1,Warmth_tree_2,Warmth_tree_3,Warmth_tree_4,Warmth_tree_5,Warmth_tree_6,Warmth_tree_7,Warmth_tree_8,Warmth_tree_9,Warmth_tree_10,Warmth_tree_11,Warmth_tree_12,Warmth_tree_13,Warmth_tree_14,Warmth_tree_15,Warmth_tree_16,Warmth_tree_17,Warmth_tree_18,Warmth_tree_19,Warmth_tree_20,Warmth_tree_21,Warmth_tree_22,Warmth_tree_23,Warmth_tree_24,Warmth_tree_25,Warmth_tree_26,Warmth_tree_27,Warmth_tree_28,Warmth_tree_29,Warmth_tree_30,Warmth_tree_31,Warmth_tree_32,Warmth_tree_33,Warmth_tree_34,Warmth_tree_35,Warmth_tree_36,Warmth_tree_37,Warmth_tree_38,Warmth_tree_39,Warmth_tree_40,Warmth_tree_41,Warmth_tree_42,Warmth_tree_43,Warmth_tree_44,Warmth_tree_45,Warmth_tree_46,Warmth_tree_47,Warmth_tree_48,Warmth_tree_49,Warmth_tree_50,Warmth_tree_51,Warmth_tree_52,Warmth_tree_53,Warmth_tree_54,Warmth_tree_55,Warmth_tree_56,Warmth_tree_57,Warmth_tree_58,Warmth_tree_59,Warmth_tree_60,Warmth_tree_61,Warmth_tree_62,Warmth_tree_63,Warmth_tree_64,Warmth_tree_65,Warmth_tree_66,Warmth_tree_67,Warmth_tree_68,Warmth_tree_69,Warmth_tree_70,Warmth_tree_71,Warmth_tree_72,Warmth_tree_73,Warmth_tree_74,Warmth_tree_75,Warmth_tree_76,Warmth_tree_77,Warmth_tree_78,Warmth_tree_79,Warmth_tree_80,Warmth_tree_81,Warmth_tree_82,Warmth_tree_83,Warmth_tree_84,Warmth_tree_85,Warmth_tree_86,Warmth_tree_87,Warmth_tree_88,Warmth_tree_89,Warmth_tree_90,Warmth_tree_91,Warmth_tree_92,Warmth_tree_93,Warmth_tree_94,Warmth_tree_95,Warmth_tree_96,Warmth_tree_97,Warmth_tree_98,Warmth_tree_99,Job Description spacy_tokenized,Job Description nltk_tokenized,Job Description gensim_tokenized,Job Description bert_tokenized,Competence_labeled_predicted,Competence_aggr_labeled_predicted,Competence_tree_0,Competence_tree_1,Competence_tree_2,Competence_tree_3,Competence_tree_4,Competence_tree_5,Competence_tree_6,Competence_tree_7,Competence_tree_8,Competence_tree_9,Competence_tree_10,Competence_tree_11,Competence_tree_12,Competence_tree_13,Competence_tree_14,Competence_tree_15,Competence_tree_16,Competence_tree_17,Competence_tree_18,Competence_tree_19,Competence_tree_20,Competence_tree_21,Competence_tree_22,Competence_tree_23,Competence_tr

In [24]:
cols_dict['Warmth']['df_jobs_unlabeled'].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303494 entries, 0 to 303493
Columns: 181 entries, Search Keyword to Warmth_tree_99
dtypes: category(2), float64(158), int64(2), object(19)
memory usage: 415.0+ MB


In [25]:
cols_dict['Competence']['df_jobs_unlabeled'].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303494 entries, 0 to 303493
Columns: 181 entries, Search Keyword to Competence_tree_99
dtypes: category(2), float64(158), int64(2), object(19)
memory usage: 415.0+ MB


In [26]:
df_jobs_unlabeled = cols_dict['Warmth']['df_jobs_unlabeled']\
    .drop(columns=list_columns)\
        .merge(
            cols_dict['Competence']['df_jobs_unlabeled'],
            how='outer',
            on=non_list_columns
        ).dropna(axis='columns', how='all')\
            .reset_index(drop=True)


In [27]:
df_jobs_unlabeled.head()


,Search Keyword,Platform,Job ID,Job Title,Company Name,Location,Dutch Requirement in Job Ad,English Requirement in Job Ad,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Sector Code,Sector,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description spacy_sentencized_lower,Dutch Requirement in Sentence,English Requirement in Sentence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Job Description spacy_sentencized_cleaned,Warmth,Warmth_Probability,Competence,Competence_Probability,Warmth_unlabeled_predicted,Warmth_aggr_unlabeled_predicted,Warmth_tree_0,Warmth_tree_1,Warmth_tree_2,Warmth_tree_3,Warmth_tree_4,Warmth_tree_5,Warmth_tree_6,Warmth_tree_7,Warmth_tree_8,Warmth_tree_9,Warmth_tree_10,Warmth_tree_11,Warmth_tree_12,Warmth_tree_13,Warmth_tree_14,Warmth_tree_15,Warmth_tree_16,Warmth_tree_17,Warmth_tree_18,Warmth_tree_19,Warmth_tree_20,Warmth_tree_21,Warmth_tree_22,Warmth_tree_23,Warmth_tree_24,Warmth_tree_25,Warmth_tree_26,Warmth_tree_27,Warmth_tree_28,Warmth_tree_29,Warmth_tree_30,Warmth_tree_31,Warmth_tree_32,Warmth_tree_33,Warmth_tree_34,Warmth_tree_35,Warmth_tree_36,Warmth_tree_37,Warmth_tree_38,Warmth_tree_39,Warmth_tree_40,Warmth_tree_41,Warmth_tree_42,Warmth_tree_43,Warmth_tree_44,Warmth_tree_45,Warmth_tree_46,Warmth_tree_47,Warmth_tree_48,Warmth_tree_49,Warmth_tree_50,Warmth_tree_51,Warmth_tree_52,Warmth_tree_53,Warmth_tree_54,Warmth_tree_55,Warmth_tree_56,Warmth_tree_57,Warmth_tree_58,Warmth_tree_59,Warmth_tree_60,Warmth_tree_61,Warmth_tree_62,Warmth_tree_63,Warmth_tree_64,Warmth_tree_65,Warmth_tree_66,Warmth_tree_67,Warmth_tree_68,Warmth_tree_69,Warmth_tree_70,Warmth_tree_71,Warmth_tree_72,Warmth_tree_73,Warmth_tree_74,Warmth_tree_75,Warmth_tree_76,Warmth_tree_77,Warmth_tree_78,Warmth_tree_79,Warmth_tree_80,Warmth_tree_81,Warmth_tree_82,Warmth_tree_83,Warmth_tree_84,Warmth_tree_85,Warmth_tree_86,Warmth_tree_87,Warmth_tree_88,Warmth_tree_89,Warmth_tree_90,Warmth_tree_91,Warmth_tree_92,Warmth_tree_93,Warmth_tree_94,Warmth_tree_95,Warmth_tree_96,Warmth_tree_97,Warmth_tree_98,Warmth_tree_99,Job Description spacy_tokenized,Job Description nltk_tokenized,Job Description gensim_tokenized,Job Description bert_tokenized,Competence_unlabeled_predicted,Competence_aggr_unlabeled_predicted,Competence_tree_0,Competence_tree_1,Competence_tree_2,Competence_tree_3,Competence_tree_4,Competence_tree_5,Competence_tree_6,Competence_tree_7,Competence_tree_8,Competence_tree_9,Competence_tree_10,Competence_tree_11,Competence_tree_12,Competence_tree_13,Competence_tree_14,Competence_tree_15,Competence_tree_16,Competence_tree_17,Competence_tree_18,Competence_tree_19,Competence_tree_20,Competence_tree_21,Competence_tree_22,Competence_tree_23,Competence_tree_24,Competence_tree_25,Competence_tree_26,Competence_tree_27,Competence_tree_28,Competence_tree_29,Competence_tree_30,Comp

In [28]:
df_jobs_unlabeled.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303494 entries, 0 to 303493
Columns: 277 entries, Search Keyword to Competence_tree_99
dtypes: category(2), float64(254), int64(2), object(19)
memory usage: 637.3+ MB


In [29]:
train = cols_dict['train']


In [30]:
train.head()


,Search Keyword,Platform,Job ID,Job Title,Company Name,Location,Dutch Requirement in Job Ad,English Requirement in Job Ad,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Sector Code,Sector,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description spacy_sentencized_lower,Dutch Requirement in Sentence,English Requirement in Sentence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Job Description spacy_tokenized,Job Description spacy_sentencized_cleaned,Job Description nltk_tokenized,Job Description gensim_tokenized,Job Description bert_tokenized,Warmth,Warmth_Probability,Competence,Competence_Probability,Warmth_actual,Competence_actual,Warmth_predicted,Warmth_Probability_predicted,Competence_predicted,Competence_Probability_predicted
291435,career development specialist,Indeed,p_7512201584b7e36b,EMEA Service Delivery Manager – DevOps,Palo Alto Networks,Amsterdam,No,No,1.00,0.00,1.00,0.00,N,Renting and other business support,8.00,416.00,42.75,3.46,1.65,557.00,57.25,4.20,2.20,Mixed Gender,311.00,31.96,2.93,1.23,660.00,67.83,4.51,2.61,Younger,973.00,3.85,0.00,0.00,1.00,0.00,0.00,1.00,1.00,2.00,1366.56,2900.09,1829.74,3883.05,1.00,0.00,1.00,0.00,Enable your team to identify gaps and develop ...,23.00,19.00,152.00,128.00,1.00,enable your team to identify gaps and develop ...,No,No,1.00,0.00,1.00,0.00,"[enable, your, team, to, identify, gaps, and, ...",enable your team to identify gaps and develop ...,"[enable, team, identify, gaps, develop, correc...","[enabl, team, identifi, gap, develop, correct,...","[enable, your, team, to, identify, gaps, and, ...",1,0.93,0,0.68,1.00,0.00,1.00,0.93,1.00,0.68
3210,specialised business service,Indeed,p_2d7b59d35856ba71,Sr Managing Counsel - Payments,Bookingcom,Amsterdam,No,No,1.00,0.00,1.00,0.00,N,Renting and other business support,8.00,416.00,42.75,3.46,1.65,557.00,57.25,4.20,2.20,Mixed Gender,311.00,31.96,2.93,1.23,660.00,67.83,4.51,2.61,Younger,973.00,3.85,0.00,0.00,1.00,0.00,0.00,1.00,1.00,2.00,1366.56,2900.09,1829.74,3883.05,1.00,0.00,1.00,0.00,8+ years hands-on experience in a similar posi...,18.00,16.00,106.00,85.00,0.00,8+ years hands-on experience in a similar posi...,No,No,1.00,0.00,1.00,0.00,"[8, +, years, hands, on, experience, in, a, si...",8 + years hands on experience in a similar pos...,"[8+, years, hands-on, experience, similar, pos...","[year, hand, experi, similar, posit, master, d...","[8, +, years, hands, -, on, experience, in, a,...",0,0.01,1,0.85,0.00,1.00,0.00,0.01,1.00,0.85
2723,specialised business service,Indeed,p_cda8e5ebf88b096f,Sr Managing Counsel - Product,Bookingcom,Amsterdam,No,No,1.00,0.00,1.00,0.00,N,Renting and other business support,8.00,416.00,42.75,3.46,1.65,557.00,57.25,4.20,2.20,Mixed Gender,311.00,31.96,2.93,1.23,660.00,67.83,4.51,2.61,Younger,973.00,3.85,0.00,0.00,1.00,0.00,0.00,1.00,1.0

In [31]:
df_jobs_train = train.copy()


In [32]:
df_jobs_train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 4458 entries, 291435 to 49280
Data columns (total 79 columns):
 #   Column                                                              Non-Null Count  Dtype   
---  ------                                                              --------------  -----   
 0   Search Keyword                                                      4458 non-null   object  
 1   Platform                                                            4458 non-null   object  
 2   Job ID                                                              4458 non-null   object  
 3   Job Title                                                           4458 non-null   object  
 4   Company Name                                                        4458 non-null   object  
 5   Location                                                            4458 non-null   object  
 6   Dutch Requirement in Job Ad                                         4458 non-null   object  
 7   Engli

In [33]:
test = cols_dict['test']


In [34]:
test.head()


,Search Keyword,Platform,Job ID,Job Title,Company Name,Location,Dutch Requirement in Job Ad,English Requirement in Job Ad,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Sector Code,Sector,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description spacy_sentencized_lower,Dutch Requirement in Sentence,English Requirement in Sentence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Job Description spacy_tokenized,Job Description spacy_sentencized_cleaned,Job Description nltk_tokenized,Job Description gensim_tokenized,Job Description bert_tokenized,Warmth,Warmth_Probability,Competence,Competence_Probability,Warmth_actual,Competence_actual,Warmth_predicted,Warmth_Probability_predicted,Competence_predicted,Competence_Probability_predicted
143571,application developer,Indeed,p_2c16ae07cd04d378,Frontend Developer (Javascript),Plan to Meet,Amsterdam,No,No,1.00,0.00,1.00,0.00,J,Information and communication,9.00,80.00,27.59,0.67,0.32,210.00,72.41,1.58,0.83,Male,95.00,32.76,0.89,0.38,195.00,67.24,1.33,0.77,Mixed Age,290.00,1.15,0.00,1.00,0.00,1.00,0.00,0.00,2.00,1.00,903.69,1854.93,2372.18,4869.20,1.00,0.00,1.00,0.00,"Script and its rich ecosystem, (including Type",7.00,7.00,46.00,38.00,0.00,"script and its rich ecosystem, (including type",No,No,1.00,0.00,1.00,0.00,"[script, and, its, rich, ecosystem, including,...",script and its rich ecosystem including type,"[script, rich, ecosystem, including, type]","[script, rich, ecosystem, includ, type]","[script, and, its, rich, ecosystem, ,, (, incl...",0,0.02,1,0.78,0.00,1.00,0.00,0.02,1.00,0.78
309257,specialised business service,Indeed,p_d55b54baef7547a7,Recruitment Business Partner (Banking),Randstad Sourceright,Amsterdam,No,No,1.00,0.00,1.00,0.00,N,Renting and other business support,8.00,416.00,42.75,3.46,1.65,557.00,57.25,4.20,2.20,Mixed Gender,311.00,31.96,2.93,1.23,660.00,67.83,4.51,2.61,Younger,973.00,3.85,0.00,0.00,1.00,0.00,0.00,1.00,1.00,2.00,1366.56,2900.09,1829.74,3883.05,1.00,0.00,1.00,0.00,Path (pr),2.00,2.00,9.00,6.00,0.00,path (pr),No,No,1.00,0.00,1.00,0.00,"[path, pr]",path pr,"[path, pr]",[path],"[path, (, pr, )]",0,0.00,1,0.63,0.00,1.00,0.00,0.00,1.00,0.63
309311,career development specialist,Indeed,p_6214f410c0ea72d0,Customer Service Specialist- Hebrew,Bookingcom,Amsterdam,No,No,1.00,0.00,1.00,0.00,N,Renting and other business support,8.00,416.00,42.75,3.46,1.65,557.00,57.25,4.20,2.20,Mixed Gender,311.00,31.96,2.93,1.23,660.00,67.83,4.51,2.61,Younger,973.00,3.85,0.00,0.00,1.00,0.00,0.00,1.00,1.00,2.00,1366.56,2900.09,1829.74,3883.05,1.00,0.00,1.00,0.00,What youll be doing:,4.00,4.00,20.00,16.00,1.00,what youll be doing:,No,No,1.00,0.00,1.00,0.00,"[what, you, ll, be, doing]",what you ll be doing,[youll],[youll],"[what, you, ##ll, be, doing, :]",0,0.01,0,0.01,0.00,0.00,0.00,0.01,0.00,0.01
217121,electricity,

In [35]:
df_jobs_test = test\
    .drop(columns=list_columns)\
    .merge(
        df_jobs_labeled,
        how='inner',
        on=non_list_columns
    ).reset_index(drop=True)



In [36]:
df_jobs_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1486 entries, 0 to 1485
Columns: 283 entries, Search Keyword to Competence_tree_99
dtypes: category(2), float64(260), int64(2), object(19)
memory usage: 3.2+ MB


In [37]:
df_jobs_test.head()


,Search Keyword,Platform,Job ID,Job Title,Company Name,Location,Dutch Requirement in Job Ad,English Requirement in Job Ad,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Sector Code,Sector,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description spacy_sentencized_lower,Dutch Requirement in Sentence,English Requirement in Sentence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Job Description spacy_sentencized_cleaned,Warmth,Warmth_Probability,Competence,Competence_Probability,Warmth_actual,Competence_actual,Warmth_predicted,Warmth_Probability_predicted,Competence_predicted,Competence_Probability_predicted,Warmth_labeled_predicted,Warmth_aggr_labeled_predicted,Warmth_tree_0,Warmth_tree_1,Warmth_tree_2,Warmth_tree_3,Warmth_tree_4,Warmth_tree_5,Warmth_tree_6,Warmth_tree_7,Warmth_tree_8,Warmth_tree_9,Warmth_tree_10,Warmth_tree_11,Warmth_tree_12,Warmth_tree_13,Warmth_tree_14,Warmth_tree_15,Warmth_tree_16,Warmth_tree_17,Warmth_tree_18,Warmth_tree_19,Warmth_tree_20,Warmth_tree_21,Warmth_tree_22,Warmth_tree_23,Warmth_tree_24,Warmth_tree_25,Warmth_tree_26,Warmth_tree_27,Warmth_tree_28,Warmth_tree_29,Warmth_tree_30,Warmth_tree_31,Warmth_tree_32,Warmth_tree_33,Warmth_tree_34,Warmth_tree_35,Warmth_tree_36,Warmth_tree_37,Warmth_tree_38,Warmth_tree_39,Warmth_tree_40,Warmth_tree_41,Warmth_tree_42,Warmth_tree_43,Warmth_tree_44,Warmth_tree_45,Warmth_tree_46,Warmth_tree_47,Warmth_tree_48,Warmth_tree_49,Warmth_tree_50,Warmth_tree_51,Warmth_tree_52,Warmth_tree_53,Warmth_tree_54,Warmth_tree_55,Warmth_tree_56,Warmth_tree_57,Warmth_tree_58,Warmth_tree_59,Warmth_tree_60,Warmth_tree_61,Warmth_tree_62,Warmth_tree_63,Warmth_tree_64,Warmth_tree_65,Warmth_tree_66,Warmth_tree_67,Warmth_tree_68,Warmth_tree_69,Warmth_tree_70,Warmth_tree_71,Warmth_tree_72,Warmth_tree_73,Warmth_tree_74,Warmth_tree_75,Warmth_tree_76,Warmth_tree_77,Warmth_tree_78,Warmth_tree_79,Warmth_tree_80,Warmth_tree_81,Warmth_tree_82,Warmth_tree_83,Warmth_tree_84,Warmth_tree_85,Warmth_tree_86,Warmth_tree_87,Warmth_tree_88,Warmth_tree_89,Warmth_tree_90,Warmth_tree_91,Warmth_tree_92,Warmth_tree_93,Warmth_tree_94,Warmth_tree_95,Warmth_tree_96,Warmth_tree_97,Warmth_tree_98,Warmth_tree_99,Job Description spacy_tokenized,Job Description nltk_tokenized,Job Description gensim_tokenized,Job Description bert_tokenized,Competence_labeled_predicted,Competence_aggr_labeled_predicted,Competence_tree_0,Competence_tree_1,Competence_tree_2,Competence_tree_3,Competence_tree_4,Competence_tree_5,Competence_tree_6,Competence_tree_7,Competence_tree_8,Competence_tree_9,Competence_tree_10,Competence_tree_11,Competence_tree_12,Competence_tree_13,Competence_tree_14,Competence_tree_15,Competence_tree_16,Competence_tree_17,Competence_tree_18,Competence_tree_19,Competence_tree_20,Competence_tree_21,Competence_tree_22,Competence_tree_23,Competence_tr

# Make instrumental Variable

### Make unbiased and biased models

In [38]:
df_jobs_for_correction = pd.concat([df_jobs_labeled, df_jobs_unlabeled], axis='index')


In [39]:
df_jobs_for_correction.info()


<class 'pandas.core.frame.DataFrame'>
Index: 309438 entries, 0 to 303493
Columns: 287 entries, Search Keyword to Competence_aggr_unlabeled_predicted
dtypes: category(2), float64(264), int64(2), object(19)
memory usage: 675.8+ MB


In [40]:
df_jobs_for_correction.head()


,Search Keyword,Platform,Job ID,Job Title,Company Name,Location,Dutch Requirement in Job Ad,English Requirement in Job Ad,Dutch Requirement in Job Ad_No,Dutch Requirement in Job Ad_Yes,English Requirement in Job Ad_No,English Requirement in Job Ad_Yes,Sector Code,Sector,Keywords Count,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Interaction_Female_Older_% per Sector,Interaction_Female_Younger_% per Sector,Interaction_Male_Older_% per Sector,Interaction_Male_Younger_% per Sector,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor,Job Description spacy_sentencized,Job Description spacy_sentencized_num_words,Job Description spacy_sentencized_num_unique_words,Job Description spacy_sentencized_num_chars,Job Description spacy_sentencized_num_chars_no_whitespact_and_punt,Job Description spacy_sentencized_num_punctuations,Job Description spacy_sentencized_lower,Dutch Requirement in Sentence,English Requirement in Sentence,Dutch Requirement in Sentence_No,Dutch Requirement in Sentence_Yes,English Requirement in Sentence_No,English Requirement in Sentence_Yes,Job Description spacy_sentencized_cleaned,Warmth,Warmth_Probability,Competence,Competence_Probability,Warmth_actual,Competence_actual,Warmth_predicted,Warmth_Probability_predicted,Competence_predicted,Competence_Probability_predicted,Warmth_labeled_predicted,Warmth_aggr_labeled_predicted,Warmth_tree_0,Warmth_tree_1,Warmth_tree_2,Warmth_tree_3,Warmth_tree_4,Warmth_tree_5,Warmth_tree_6,Warmth_tree_7,Warmth_tree_8,Warmth_tree_9,Warmth_tree_10,Warmth_tree_11,Warmth_tree_12,Warmth_tree_13,Warmth_tree_14,Warmth_tree_15,Warmth_tree_16,Warmth_tree_17,Warmth_tree_18,Warmth_tree_19,Warmth_tree_20,Warmth_tree_21,Warmth_tree_22,Warmth_tree_23,Warmth_tree_24,Warmth_tree_25,Warmth_tree_26,Warmth_tree_27,Warmth_tree_28,Warmth_tree_29,Warmth_tree_30,Warmth_tree_31,Warmth_tree_32,Warmth_tree_33,Warmth_tree_34,Warmth_tree_35,Warmth_tree_36,Warmth_tree_37,Warmth_tree_38,Warmth_tree_39,Warmth_tree_40,Warmth_tree_41,Warmth_tree_42,Warmth_tree_43,Warmth_tree_44,Warmth_tree_45,Warmth_tree_46,Warmth_tree_47,Warmth_tree_48,Warmth_tree_49,Warmth_tree_50,Warmth_tree_51,Warmth_tree_52,Warmth_tree_53,Warmth_tree_54,Warmth_tree_55,Warmth_tree_56,Warmth_tree_57,Warmth_tree_58,Warmth_tree_59,Warmth_tree_60,Warmth_tree_61,Warmth_tree_62,Warmth_tree_63,Warmth_tree_64,Warmth_tree_65,Warmth_tree_66,Warmth_tree_67,Warmth_tree_68,Warmth_tree_69,Warmth_tree_70,Warmth_tree_71,Warmth_tree_72,Warmth_tree_73,Warmth_tree_74,Warmth_tree_75,Warmth_tree_76,Warmth_tree_77,Warmth_tree_78,Warmth_tree_79,Warmth_tree_80,Warmth_tree_81,Warmth_tree_82,Warmth_tree_83,Warmth_tree_84,Warmth_tree_85,Warmth_tree_86,Warmth_tree_87,Warmth_tree_88,Warmth_tree_89,Warmth_tree_90,Warmth_tree_91,Warmth_tree_92,Warmth_tree_93,Warmth_tree_94,Warmth_tree_95,Warmth_tree_96,Warmth_tree_97,Warmth_tree_98,Warmth_tree_99,Job Description spacy_tokenized,Job Description nltk_tokenized,Job Description gensim_tokenized,Job Description bert_tokenized,Competence_labeled_predicted,Competence_aggr_labeled_predicted,Competence_tree_0,Competence_tree_1,Competence_tree_2,Competence_tree_3,Competence_tree_4,Competence_tree_5,Competence_tree_6,Competence_tree_7,Competence_tree_8,Competence_tree_9,Competence_tree_10,Competence_tree_11,Competence_tree_12,Competence_tree_13,Competence_tree_14,Competence_tree_15,Competence_tree_16,Competence_tree_17,Competence_tree_18,Competence_tree_19,Competence_tree_20,Competence_tree_21,Competence_tree_22,Competence_tree_23,Competence_tr

In [41]:
# Biased model
biased_post_classification_dict = defaultdict()
for iv in tqdm.tqdm(ivs_dummy_perc_and_perc_interactions):
    dv_names_dict_unlabeled_post_classification = compare_actual_and_predicted(
        df_jobs_unlabeled, analysis_type='post_classification', iv_names=iv
    )
    biased_post_classification_dict[iv] = dv_names_dict_unlabeled_post_classification


  0%|          | 0/14 [00:00<?, ?it/s]

Processing dataframe of length 303494
Analyzing Warmth dict_keys(['Biased']) Models


Saving /Users/nyxinsane/Documents/Work - UvA/Automating Equity/Automating_Equity1/Automating_Equity1_Code/data/output tables/post_classification Biased OLS Regression Warmth...


  0%|          | 0/88 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  3.37it/s]


...

Saving /Users/nyxinsane/Documents/Work - UvA/Automating Equity/Automating_Equity1/Automating_Equity1_Code/data/output tables/post_classification Biased OLS Regression Competence...


  0%|          | 0/88 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:07<00:00,  3.68s/it]



--------------------
BIASED Competence

--------------------


BIASED SUMMARY RESULTS:
                                      OLS Regression Results                                     
Dep. Variable:     Interaction_Male_Younger_% per Sector   R-squared:                       0.001
Model:                                               OLS   Adj. R-squared:                  0.001
Method:                                    Least Squares   F-statistic:                     78.47
Date:                                   Wed, 01 Nov 2023   Prob (F-statistic):           9.74e-51
Time:                                           04:40:43   Log-Likelihood:            -2.5848e+06
No. Observations:                                 303494   AIC:                         5.170e+06
Df Residuals:                                     303490   BIC:                         5.170e+06
Df Model:                                              3                                         
Covariance Type:             


100%|██████████| 14/14 [00:44<00:00,  3.19s/it]


In [42]:
biased_post_classification_dict.keys()


dict_keys(['Gender_Female', 'Gender_Mixed', 'Gender_Male', 'Gender_Female_% per Sector', 'Gender_Male_% per Sector', 'Age_Older', 'Age_Mixed', 'Age_Younger', 'Age_Older_% per Sector', 'Age_Younger_% per Sector', 'Interaction_Female_Older_% per Sector', 'Interaction_Female_Younger_% per Sector', 'Interaction_Male_Older_% per Sector', 'Interaction_Male_Younger_% per Sector'])

In [43]:
# Uniased model
unbiased_post_classification_dict = defaultdict()
for iv in tqdm.tqdm(ivs_dummy_perc_and_perc_interactions):
    dv_names_dict_labeled_post_classification = compare_actual_and_predicted(
        df_jobs_labeled, analysis_type='post_classification', iv_names=iv
    )
    unbiased_post_classification_dict[iv] = dv_names_dict_labeled_post_classification


  0%|          | 0/14 [00:00<?, ?it/s]

Processing dataframe of length 5944
Analyzing Warmth dict_keys(['Unbiased']) Models


Saving /Users/nyxinsane/Documents/Work - UvA/Automating Equity/Automating_Equity1/Automating_Equity1_Code/data/output tables/post_classification Unbiased OLS Regression Warmth...


  0%|          | 0/88 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  1.72it/s]


...

Processing dataframe of length 5944
Analyzing Competence dict_keys(['Unbiased']) Models


Saving /Users/nyxinsane/Documents/Work - UvA/Automating Equity/Automating_Equity1/Automating_Equity1_Code/data/output tables/post_classification Unbiased OLS Regression Competence...


  0%|          | 0/88 [00:00<?, ?it/s]



--------------------
UNBIASED Competence

--------------------


UNBIASED SUMMARY RESULTS:
                                      OLS Regression Results                                     
Dep. Variable:     Interaction_Male_Younger_% per Sector   R-squared:                       0.024
Model:                                               OLS   Adj. R-squared:                  0.023
Method:                                    Least Squares   F-statistic:                     48.26
Date:                                   Wed, 01 Nov 2023   Prob (F-statistic):           8.27e-31
Time:                                           04:40:55   Log-Likelihood:                -49527.
No. Observations:                                   5944   AIC:                         9.906e+04
Df Residuals:                                       5940   BIC:                         9.909e+04
Df Model:                                              3                                         
Covariance Type:         


100%|██████████| 14/14 [00:10<00:00,  1.29it/s]


In [44]:
unbiased_post_classification_dict.keys()


dict_keys(['Gender_Female', 'Gender_Mixed', 'Gender_Male', 'Gender_Female_% per Sector', 'Gender_Male_% per Sector', 'Age_Older', 'Age_Mixed', 'Age_Younger', 'Age_Older_% per Sector', 'Age_Younger_% per Sector', 'Interaction_Female_Older_% per Sector', 'Interaction_Female_Younger_% per Sector', 'Interaction_Male_Older_% per Sector', 'Interaction_Male_Younger_% per Sector'])

In [45]:
# Get forest_iv results
forest_iv_results_dict = defaultdict(lambda: defaultdict())
forest_iv_params = {
    # 'col': dv,
    # 'var': iv,
    # 'model_unbias': model_unbias,
    'data_test': df_jobs_test,
    'data_unlabel': df_jobs_unlabeled,
    'control': controls[:2],
    'ntree': n_trees,
    # 'diagnostic': True,
    # 'family': sm.families.Gaussian(link=sm.families.links.Identity()),
    # 'select_method': 'optimal',
    # 'method': 'Lasso',
    # 'iterative': False
}

for dv, iv in tqdm_product(dvs, ivs_dummy_perc_and_perc_interactions):
    forest_iv_params['col'] = dv
    forest_iv_params['var'] = iv
    forest_iv_params['model_unbias'] = unbiased_post_classification_dict[iv][dv]['Unbiased']['Results']
    result = forest_iv(**forest_iv_params)

    forest_iv_results_dict[dv][iv] = result

# result = forest_iv(
#     col=dv,
#     data_test=df_jobs_test,
#     data_unlabel=df_jobs_unlabeled,
#     var=iv,
#     control=controls[:2],
#     ntree=n_trees,
#     model_unbias=model_unbias,
#     diagnostic=True,
#     family=sm.families.Gaussian(link=sm.families.links.Identity()),
#     select_method='optimal',
#     method='Lasso',
#     iterative=False
# )


  0%|          | 0/28 [00:00<?, ?it/s]

--------------------
Analyzing Warmth_tree_0/99 trees


--------------------
Analyzing Warmth_tree_1/99 trees


--------------------
Analyzing Warmth_tree_2/99 trees


--------------------
Analyzing Warmth_tree_3/99 trees


...

--------------------
Analyzing Warmth_tree_5/99 trees


--------------------
Analyzing Warmth_tree_6/99 trees


--------------------
Analyzing Warmth_tree_7/99 trees


--------------------
Analyzing Warmth_tree_8/99 trees


--------------------
Analyzing Warmth_tree_9/99 trees


In [ ]:
forest_iv_results_dict.keys()


In [ ]:
# Calculate the critical value for a chi-squared distribution
H_critical = chi2.ppf(0.95, df=4)

for dv, iv in tqdm_product(dvs, ivs_dummy_perc_and_perc_interactions):
    # Get unbiased model
    model_unbias = unbiased_post_classification_dict[iv][dv]['Unbiased']['Results']

    # Get the unbiased coefficients
    coef_unbiased = model_unbias.coef

    # Get result
    result = forest_iv_results_dict[dv][iv]

    # Calculate the squared bias for each beta
    result['bias2'] = ((result[['beta_1', 'beta_2', 'beta_3', 'beta_4']] - coef_unbiased) ** 2).sum(axis=1)

    # Calculate the total variance
    result['variance'] = (result[['se_1', 'se_2', 'se_3', 'se_4']] ** 2).sum(axis=1)

    # Calculate the mean squared error (MSE)
    result['mse'] = result['bias2'] + result['variance']

    # Sort the DataFrame by MSE
    result = result.sort_values(by='mse')

    # Filter rows where Hotelling is less than H_critical and only keep the first row
    filtered_result = result[(result['Hotelling'] < H_critical) & (result.index == result.index[0])]

    # Display the filtered result
    print(filtered_result[['beta_1', 'beta_2', 'beta_3', 'beta_4']])


In [ ]:
# # Calculate the critical value of Hotelling's T-squared test
# H_critical = chi2.ppf(0.95, df=4)

# # Get the unbiased coefficients
# coef_unbiased = model_unbias.coef

# # Calculate the bias squared, variance, and mean squared error (MSE)
# bias2 = np.sum((coef_unbiased - [beta_1, beta_2, beta_3, beta_4])**2)
# variance = se_1**2 + se_2**2 + se_3**2 + se_4**2
# mse = bias2 + variance

# # Add these columns to the `result` DataFrame
# result = result.assign(
#     bias2=bias2,
#     variance=variance,
#     mse=mse,
# )

# # Sort the DataFrame by MSE and filter to the top row
# result = result.sort_values("mse").iloc[:1]

# # Filter to the rows where Hotelling's T-squared test is less than the critical value
# result = result.query("Hotelling < {}".format(H_critical))

# # Print the results
# print(result)


In [ ]:
# HACK
# def compute_embeddings(model, input_ids):
#     outputs = model(input_ids)
#     hidden_states = outputs.hidden_states
#     embeddings = hidden_states[-1]  # Extract embeddings from the last layer
#     return embeddings

# train_data = estimator.get_train_dataloader()
# eval_data = estimator.get_eval_dataloader()

# # Compute embeddings for your train and eval data
# train_embeddings = compute_embeddings(model, next(iter(train_data))[0])
# eval_embeddings = compute_embeddings(model, next(iter(eval_data))[0])

# TODO: get train, test, datasets from transformers save folder, X = np.concatenate((X_test, X_val), axis=0) and y = np.concatenate((y_test, y_val), axis=0) so X_test, y_test will be both of these. Get these to become df_jobs_test and df_jobs_train, then df_jobs_unlabeled will be the same. 

# from transformers import BertModel, Trainer

# model = BertModel.from_pretrained("bert-base-uncased")
# trainer = Trainer(model)

# trainer.train()

# # Get the embeddings from the model
# embeddings = model.get_input_embeddings()

# print(embeddings.shape)

# from transformers import BertModel, Trainer

# model = BertModel.from_pretrained("bert-base-uncased")
# trainer = Trainer(model)

# trainer.train()

# # Get the hidden states from the model
# hidden_states = model.get_hidden_states()

# # Get the embeddings from the last layer
# embeddings = hidden_states[-1]

# print(embeddings.shape)
